In [1]:
import torch
torch.__version__

'1.7.1'

To start using zoo.orca, we need to first initialize orca context. Here we specify local or distributed mode. In this example, we choose the local mode.

In [ ]:
from zoo.orca import init_orca_context, stop_orca_context
from zoo.orca import OrcaContext

# recommended to set it to True when running Analytics Zoo in Jupyter notebook. 
OrcaContext.log_output = True # (this will display terminal's stdout and stderr in the Jupyter notebook).

cluster_mode = "local"

if cluster_mode == "local":
    init_orca_context(cores=1, memory="2g")   # run in local mode
elif cluster_mode == "k8s":
    init_orca_context(cluster_mode="k8s", num_nodes=2, cores=4) # run on K8s cluster
elif cluster_mode == "yarn":
    init_orca_context(
        cluster_mode="yarn-client", cores=4, num_nodes=2, memory="2g",
        driver_memory="10g", driver_cores=1,
        conf={"spark.rpc.message.maxSize": "1024",
              "spark.task.maxFailures": "1",
              "spark.driver.extraJavaOptions": "-Dbigdl.failure.retryTimes=1"})   # run on Hadoop YARN cluster

# Text generation with LSTM
This notebook contains the code samples found in Chapter 8, Section 1 of Deep Learning with Python. Note that the original text features far more content, in particular further explanations and figures: in this notebook, you will only find source code and related comments.

-----------------------------------------

[...]

## Implementing character-level LSTM text generation
Let's put these ideas in practice in a Keras implementation. The first thing we need is a lot of text data that we can use to learn a language model. You could use any sufficiently large text file or set of text files -- Wikipedia, the Lord of the Rings, etc. In this example we will use some of the writings of Nietzsche, the late-19th century German philosopher (translated to English). The language model we will learn will thus be specifically a model of Nietzsche's writing style and topics of choice, rather than a more generic model of the English language.

## Preparing the data
Let's start by downloading the corpus and converting it to lowercase:

In [3]:
from torchvision.datasets.utils import download_url

download_url("https://s3.amazonaws.com/text-datasets/nietzsche.txt", '.')
text = open('./nietzsche.txt').read().lower()
print('Corpus length:', len(text))

Using downloaded and verified file: ./nietzsche.txt
Corpus length: 600893


Next, we will extract partially-overlapping sequences of length maxlen, one-hot encode them and pack them in a 3D Numpy array `x` of shape `(sequences, maxlen, unique_characters)`. Simultaneously, we prepare a array `y` containing the corresponding targets: the one-hot encoded characters that come right after each extracted sequence.

In [4]:
import numpy as np

# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 20

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen), dtype=np.int32)
y = np.zeros((len(sentences)), dtype=np.int32)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t] = char_indices[char]
    y[i] = char_indices[next_chars[i]]

Number of sequences: 30042
Unique characters: 57
Vectorization...


In [5]:
x.shape

(30042, 60)

In [6]:
y.shape

(30042,)


## Building the network
Our network is a single LSTM layer followed by a Dense classifier and softmax over all possible characters. But let us note that recurrent neural networks are not the only way to do sequence data generation; 1D convnets also have proven extremely successful at it in recent times. Since our targets are one-hot encoded, we will use `categorical_crossentropy` as the loss to train the model:

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LSTMNet(nn.Module):
    def __init__(self, maxlen, chars):
        super(LSTMNet, self).__init__()
        self.embedding = nn.Embedding(len(chars), maxlen)
        self.lstm = nn.LSTM(maxlen, 128)
        self.fc = nn.Linear(128, len(chars))
        
        self.state_h = torch.randn(1, maxlen, 128)
        self.state_c = torch.randn(1, maxlen, 128)

    def forward(self, x):
        batch_size, vocab_size = x.size()
        x = self.embedding(x)
        (x, (h, c)) = self.lstm(x, (self.state_h, self.state_c))
        self.state_h = h.detach()
        self.state_c = c.detach()
        return F.log_softmax(self.fc(x[:, -1, :]), dim=-1)
        
def model_creator(config):
    model = LSTMNet(maxlen, chars)
    return model

criterion = nn.CrossEntropyLoss()

def optim_creator(model, config):
    return torch.optim.Adam(model.parameters(), lr=0.001)

In [8]:
model = model_creator(config={})
print(model)

LSTMNet(
  (embedding): Embedding(57, 60)
  (lstm): LSTM(60, 128)
  (fc): Linear(in_features=128, out_features=57, bias=True)
)


We also create a `data_creator` function that help us import the data into the orca estimator.

In [9]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

def data_loader_creator(data, targets, batch_size, shuffle=True):
    """
    Transforms data and targets from np.array to torch.utils.data.DataLoader,
    and shuffle the dataset if `shuffle` is set to true.
    """
    
    data_tensor = torch.LongTensor(data.copy())
    targets_tensor = torch.LongTensor(targets.copy())
    dataset = TensorDataset(data_tensor, targets_tensor)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

In [10]:
def train_data_creator(config, batch_size):
    return data_loader_creator(x, y, batch_size, shuffle=True)

Training the language model and sampling from it
Given a trained model and a seed text snippet, we generate new text by repeatedly:

* 1) Drawing from the model a probability distribution over the next character given the text available so far
* 2) Reweighting the distribution to a certain "temperature"
* 3) Sampling the next character at random according to the reweighted distribution
* 4) Adding the new character at the end of the available text

This is the code we use to reweight the original probability distribution coming out of the model, and draw a character index from it (the "sampling function"):

In [11]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = preds / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Finally, this is the loop where we repeatedly train and generated text. We start generating text using a range of different temperatures after every epoch. This allows us to see how the generated text evolves as the model starts converging, as well as the impact of temperature in the sampling strategy.

In [ ]:
from zoo.orca.learn.pytorch import Estimator 
from zoo.orca.learn.metrics import Accuracy

est = Estimator.from_torch(model=model_creator, optimizer=optim_creator, loss=criterion, 
                           backend="torch_distributed")

In [13]:
import random
import time

batch_size=128
output = ""
for epoch in range(1, 60):
    output += "epoch " + str(epoch) + '\n'
    # Fit the model for 1 epoch on the available training data
    stats = est.fit(data=train_data_creator, epochs=1, batch_size=128)
    for epochinfo in stats:
        print("===> Epoch {} Complete: Avg. Loss: {:.4f}"
              .format(epoch, epochinfo["train_loss"]))
    elapsed = time.time() - t
    print("training used time", elapsed)
    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    output += '--- Generating with seed: "' + str(generated_text) + '"\n'
    trained_model = est.get_model()
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        output += '------ temperature: ' + str(temperature) + '\n'
        output += generated_text

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen))
            for t, char in enumerate(generated_text):
                sampled[0, t] = char_indices[char]
            
            next_index = sample(trained_model(torch.LongTensor(sampled)).detach().numpy()[0], temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]
            output += next_char
            
        output += '\n'

(pid=16522) /home/zhenhao/.local/lib/python3.7/site-packages/torch/autograd/__init__.py:132: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
(pid=16522)   allow_unreachable=True)  # allow_unreachable flag
===> Epoch 1 Complete: Avg. Loss: 2.7916
training used time 27.625569105148315
===> Epoch 2 Complete: Avg. Loss: 2.5292
training used time 27.599589586257935
===> Epoch 3 Complete: Avg. Loss: 2.4902
training used time 26.805781841278076
===> Epoch 4 Complete: Avg. Loss: 2.4703
training used time 27.479182481765747
===> Epoch 5 Complete: Avg. Loss: 2.4569
training used time 27.252915859222412
===> Epoch 6 Complete: Avg. Loss: 2.4462
training used time 27.035186529159546
===> Epoch 7 Complete: Avg. Loss: 2.4368
training used time 26.974387645721436
===> Epoch 8 Complete: Avg. Loss: 2.42

In [14]:
print(output)

epoch 1
--- Generating with seed: "lusion of volition. if, on a sudden, the entire movement of "
------ temperature: 0.2
lusion of volition. if, on a sudden, the entire movement of athe an ind and the an the an t the ang the the and the an the an the an an the athe an ore an whe an the on the ind athe on and t s t the t the an the the an the and the the ore the the an t the an s t an the the on the an the the the an ore the and the t the an in an t ithe an the the the the an the in the and an the an the the and t the the in the the the in athe ang an and the an an the an is 
------ temperature: 0.5
e the in the the the in athe ang an and the an an the an is d he ined be h se ase ace ances in alof ore onouren wibed
ange is pe the t tilo the athe we thes ans thes wowhereuof ty tithere whe therofor
tire ar ton ade thest thance re t thend as se the ore t at angant ancind is onane thins ore an the che asond the t s d on the rure sy e ce t g ene t inthe there t sthe ane in ind achent ale ce 

As you can see, a low temperature results in extremely repetitive and predictable text, but where local structure is highly realistic: in particular, all words (a word being a local pattern of characters) are real English words. With higher temperatures, the generated text becomes more interesting, surprising, even creative; it may sometimes invent completely new words that sound somewhat plausible (such as "eterned" or "troveration"). With a high temperature, the local structure starts breaking down and most words look like semi-random strings of characters. Without a doubt, here 0.5 is the most interesting temperature for text generation in this specific setup. Always experiment with multiple sampling strategies! A clever balance between learned structure and randomness is what makes generation interesting.

Note that by training a bigger model, longer, on more data, you can achieve generated samples that will look much more coherent and realistic than ours. But of course, don't expect to ever generate any meaningful text, other than by random chance: all we are doing is sampling data from a statistical model of which characters come after which characters. Language is a communication channel, and there is a distinction between what communications are about, and the statistical structure of the messages in which communications are encoded. To evidence this distinction, here is a thought experiment: what if human language did a better job at compressing communications, much like our computers do with most of our digital communications? Then language would be no less meaningful, yet it would lack any intrinsic statistical structure, thus making it impossible to learn a language model like we just did.

## Take aways
* We can generate discrete sequence data by training a model to predict the next tokens(s) given previous tokens.
* In the case of text, such a model is called a "language model" and could be based on either words or characters.
* Sampling the next token requires balance between adhering to what the model judges likely, and introducing randomness.
* One way to handle this is the notion of softmax temperature. Always experiment with different temperatures to find the "right" one.

In [15]:
stop_orca_context()

Stopping orca context
